In [1]:
import os
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

In [2]:
df = pd.read_csv("data/train_set.csv", sep="\t", header=0)
df

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
...,...,...
199995,2,307 4894 7539 4853 5330 648 6038 4409 3764 603...
199996,2,3792 2983 355 1070 4464 5050 6298 3782 3130 68...
199997,11,6811 1580 7539 1252 1899 5139 1386 3870 4124 1...
199998,2,6405 3203 6644 983 794 1913 1678 5736 1397 191...


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
dataset = Dataset.from_pandas(df).shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.2)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 160000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 40000
    })
})


In [5]:
max_len = 512
def map_func(examples):
    # examples 还是个 dict
    result = {"input_ids": [], "token_type_ids": [], "attention_mask": []}
    for text in examples["text"]:
        input_ids = [tokenizer.cls_token_id] + [int(x) for x in text.strip().split(" ")][:max_len-2] + [tokenizer.sep_token_id]
        input_len = len(input_ids)
        token_type_ids = [0] * max_len
        attention_mask = [1] * input_len + [0] * (max_len - input_len)
        input_ids = input_ids + [0] * (max_len - input_len)

        result["input_ids"].append(input_ids)
        result["token_type_ids"].append(token_type_ids)
        result["attention_mask"].append(attention_mask)
    if "label" in examples:
        result["label"] = examples["label"]
    return result

In [6]:
train_dataset = dataset["train"].map(map_func, batched=True)
eval_dataset = dataset["test"].map(map_func, batched=True)

100%|██████████| 40/40 [00:06<00:00,  6.41ba/s]


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=14)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
# 训练参数
model_path = "tianchi_model"
os.makedirs(model_path, exist_ok=True)
training_args = TrainingArguments(
    output_dir=model_path,
    evaluation_strategy="steps",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=128,
    logging_steps=500,
    save_steps=500,
    save_total_limit=10,
    load_best_model_at_end=True,
    warmup_steps=3000,
    weight_decay=0.01,
    learning_rate = 1e-4,
    report_to="tensorboard",
    fp16=True,
)

In [9]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
		# 需要一个元组, 第一个是 logits, 第二个是标签
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Using the latest cached version of the module from C:\Users\zhenh\.cache\huggingface\modules\datasets_modules\metrics\accuracy\3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Sat Jan 15 11:26:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Using amp half precision backend


In [11]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 160000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 4
  Total optimization steps = 12500
  4%|▍         | 500/12500 [07:27<2:58:49,  1.12it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 128


{'loss': 1.7407, 'learning_rate': 1.6633333333333336e-05, 'epoch': 0.2}


                                                     
  4%|▍         | 500/12500 [10:32<2:58:49,  1.12it/s]Saving model checkpoint to tianchi_model\checkpoint-500
Configuration saved in tianchi_model\checkpoint-500\config.json


{'eval_loss': 0.916534960269928, 'eval_accuracy': 0.7331, 'eval_runtime': 184.1196, 'eval_samples_per_second': 217.25, 'eval_steps_per_second': 1.7, 'epoch': 0.2}


Model weights saved in tianchi_model\checkpoint-500\pytorch_model.bin
  8%|▊         | 1000/12500 [17:58<2:50:37,  1.12it/s] The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 128


...

                                                       
100%|██████████| 12500/12500 [4:31:41<00:00,  1.19it/s]Saving model checkpoint to tianchi_model\checkpoint-12500
Configuration saved in tianchi_model\checkpoint-12500\config.json


{'eval_loss': 0.18184685707092285, 'eval_accuracy': 0.957775, 'eval_runtime': 181.2177, 'eval_samples_per_second': 220.729, 'eval_steps_per_second': 1.727, 'epoch': 5.0}


Model weights saved in tianchi_model\checkpoint-12500\pytorch_model.bin
Deleting older checkpoint [tianchi_model\checkpoint-7500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from tianchi_model\checkpoint-9500 (score: 0.162276029586792).
100%|██████████| 12500/12500 [4:31:43<00:00,  1.30s/it]

{'train_runtime': 16303.3632, 'train_samples_per_second': 49.07, 'train_steps_per_second': 0.767, 'train_loss': 0.23910002365112304, 'epoch': 5.0}


TrainOutput(global_step=12500, training_loss=0.23910002365112304, metrics={'train_runtime': 16303.3632, 'train_samples_per_second': 49.07, 'train_steps_per_second': 0.767, 'train_loss': 0.23910002365112304, 'epoch': 5.0})

In [12]:
trainer.save_model(model_path)

Configuration saved in tianchi_model\config.json
Model weights saved in tianchi_model\pytorch_model.bin


In [15]:
# 预测
test_df = pd.read_csv("data/test_a.csv", sep="\t", header=0)

test_dataset = Dataset.from_pandas(test_df).map(map_func, batched=True)

100%|██████████| 50/50 [00:07<00:00,  6.36ba/s]


In [33]:
predict_result = trainer.predict(test_dataset)
predict_result

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 128
1173it [28:24,  1.78it/s]

PredictionOutput(predictions=array([[ 0.7617 ,  8.65   , -2.686  , ..., -1.881  , -3.012  , -3.59   ],
       [-1.743  , -1.503  , 11.46   , ..., -0.545  ,  2.201  , -0.7485 ],
       [-0.6426 ,  0.3857 , -1.32   , ..., -0.10345, -2.34   , -0.8706 ],
       ...,
       [-0.678  ,  7.69   , -2.492  , ..., -2.531  , -1.758  , -1.978  ],
       [-0.1014 , -0.647  ,  0.5474 , ...,  1.251  , -2.979  , -2.889  ],
       [ 0.2166 ,  9.3    , -2.232  , ..., -1.683  , -2.373  , -3.08   ]],
      dtype=float16), label_ids=None, metrics={'test_runtime': 245.8173, 'test_samples_per_second': 203.403, 'test_steps_per_second': 1.591})

In [35]:
predict_df = pd.DataFrame(np.argmax(predict_result.predictions, axis=-1).tolist(), columns=["label"])
predict_df.to_csv("predict.csv", index=False)
predict_df

,label
0,1
1,2
2,8
3,5
4,0
...,...
49995,0
49996,13
49997,1
49998,3


In [26]:
tokenizer("hello", padding="max_length", truncation=True, max_length=5)

{'input_ids': [101, 7592, 102, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]}

In [32]:
map_func({"text": "12 24"})

{'input_ids': [101, 12, 24, 102, 0, 0, 0, 0, 0, 0],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [23]:
print(tokenizer.cls_token_id)
print(tokenizer.sep_token_id)

101
102
